In [102]:
import pandas as pd
import numpy as np
import warnings 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn import kernel_ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

In [103]:
data = pd.read_csv('features.csv',  engine = 'python',  on_bad_lines = 'skip', index_col=[0])

In [104]:
warnings.filterwarnings('ignore')

In [105]:
data[data.isnull().any(axis=1)]

,NumberofFloors,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),GHGEmissionsIntensity,Type_Distribution Center,Type_Hospital,...,Neighbor_NORTHEAST,Neighbor_NORTHWEST,Neighbor_North,Neighbor_Northwest,Neighbor_SOUTHEAST,Neighbor_SOUTHWEST,Built_1900-1945,Built_1946-1970,Built_1971-1990,Built_1991-2015


In [106]:
data = data.dropna(axis=0, how='any')

In [107]:
print(data.isnull().sum())

NumberofFloors                      0
ENERGYSTARScore                     0
SiteEUI(kBtu/sf)                    0
SiteEUIWN(kBtu/sf)                  0
SteamUse(kBtu)                      0
Electricity(kBtu)                   0
NaturalGas(kBtu)                    0
GHGEmissionsIntensity               0
Type_Distribution Center            0
Type_Hospital                       0
Type_Hotel                          0
Type_K-12 School                    0
Type_Laboratory                     0
Type_Large Office                   0
Type_Low-Rise Multifamily           0
Type_Medical Office                 0
Type_Mixed Use Property             0
Type_Other                          0
Type_Refrigerated Warehouse         0
Type_Residence Hall                 0
Type_Restaurant                     0
Type_Retail Store                   0
Type_Self-Storage Facility          0
Type_Senior Care Community          0
Type_Small- and Mid-Sized Office    0
Type_Supermarket / Grocery Store    0
Type_Univers

In [108]:
data.shape

(1534, 52)

# 3 Prédiction Consomation

## 3.2. préparation des données

In [109]:
# features numériques utilisées 
X_cols_num = ['NumberofFloors', 'ENERGYSTARScore', 'SiteEUIWN(kBtu/sf)', 'SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']
len(X_cols_num)

6

In [113]:
#X_cols_remove = ['GHGEmissionsIntensity', 'GHGEmissionsIntensity_log', 'SiteEUI(kBtu/sf)_log', 'SiteEUI(kBtu/sf)' ]
X_cols_remove = ['GHGEmissionsIntensity', 'SiteEUI(kBtu/sf)' ]

In [114]:
X = data.drop(X_cols_remove, axis = 1).values
y = data['SiteEUI(kBtu/sf)'].values
#y = data['SiteEUI(kBtu/sf)_log'].values

In [115]:
# séparation de train et test 
from sklearn import model_selection
X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [116]:
# normalisation seulement des variables numériques
from sklearn import preprocessing

scaler = StandardScaler()

X_train_num = X_train[:, :len(X_cols_num)]
X_train_cat = X_train[:, len(X_cols_num):]
X_train_num_std = scaler.fit_transform(X_train_num)
X_train_std = np.concatenate((X_train_num_std, X_train_cat), axis=1)

X_test_num = X_test[:, :len(X_cols_num)]
X_test_cat = X_test[:, len(X_cols_num):]
X_test_num_std = scaler.fit_transform(X_test_num)
X_test_std = np.concatenate((X_test_num_std, X_test_cat), axis=1)

In [117]:
def performance(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred)
    print("MSE : {:.2f}".format(mse))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE: {:.2f}".format(rmse))
    r2 = r2_score(y_test, y_pred)
    print("R2 : {:.2f}".format(r2))
    return mse, rmse, r2

## 3.3 Choix de modèle et noyau

### 3.3.1 Baseline avec regression linéaire 

In [118]:
model = LinearRegression()
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 498065661910381101056.00
RMSE: 22317384746.21
R2 : -119872645058118032.00


In [123]:
from sklearn.dummy import DummyRegressor
dummy_regressor = DummyRegressor(strategy='mean')

# Entraîner le dummy regressor sur les données d'entraînement
dummy_regressor.fit(X_train_std, y_train)

# Prédire les valeurs cibles pour les données de test
y_pred = dummy_regressor.predict(X_test_std)

mse, rmse, r2 = performance(y_test, y_pred)

MSE : 4199.78
RMSE: 64.81
R2 : -0.01


### 3.3.2   Modèle  avec  KernelRidgeRegression à  noyau 

In [119]:
# Noyau linéaire 
model = KernelRidge(alpha=1, kernel='linear')
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
mse, rmse, r2 = performance(y_test, y_pred)


MSE : 239.94
RMSE: 15.49
R2 : 0.94


In [120]:
# avec Noyau non linéaire gaussien rbf 
model = KernelRidge(alpha=1, kernel='rbf', gamma=0.01)
model.fit(X_train_std, y_train)
y_pred = model.predict(X_test_std)
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 202.64
RMSE: 14.24
R2 : 0.95


Recheche des hyper paramètres par grilles de alpha, gamma et noyau

In [68]:
# Create a KRR model  ( Running time = 10s)
kr = KernelRidge()

# Définir la grille de paramètres à tester
param_grid = {
    'alpha': np.logspace(-4, 2, 5),
    'gamma': np.logspace(-4, 2, 5),
    'kernel': ['linear', 'polynomial', 'rbf']
}

# Effectuer la recherche d'hyperparamètres par validation croisée et recherche en grille
grid_search = GridSearchCV(estimator=kr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train_std, y_train)

# Afficher les meilleurs paramètres trouvés et leur performance
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure performance : ", np.sqrt(-grid_search.best_score_))

# Utiliser les meilleurs paramètres pour ajuster le modèle final
kr_final = grid_search.best_estimator_

# prédire sur le jeu de test avec le modèle sélectionné 
y_pred = kr_final.predict(X_test_std)

# calculer les performance 
mse, rmse, r2 = performance(y_test, y_pred)

Meilleurs paramètres :  {'alpha': 0.0031622776601683794, 'gamma': 0.0031622776601683794, 'kernel': 'rbf'}
Meilleure performance :  0.5485871521614861
MSE : 0.35
RMSE: 0.59
R2 : 0.48


In [69]:
from sklearn.svm import SVR

# Définition du modèle SVM
svm_model = SVR(kernel='rbf', C=1)

# Entraînement du modèle SVM
svm_model.fit(X_train_std, y_train)

# Prédiction sur l'ensemble de test
y_pred = svm_model.predict(X_test_std)

# Calcul de la performance du modèle
# calculer les performance 
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 0.27
RMSE: 0.52
R2 : 0.60


### 3.3.3  Modèle ensembliste parallèle : Random forest Regression
Ramdon forest est adapté pour prédiction par regression 

In [70]:
from sklearn.ensemble import RandomForestRegressor

# Créer un modèle de forêt aléatoire
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=0)

# Former le modèle sur les données non normalisées, car ensembliste n'est pas sensible 
rf_regressor.fit(X_train, y_train)

# prédire sur le jeu de test avec le modèle sélectionné 
y_pred = rf_regressor.predict(X_test)

# calculer les performance 
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 0.22
RMSE: 0.47
R2 : 0.67


Recheche de hyper paramètres pour RF 

In [76]:
# (Running time = 36min  , with CPU 90%)
from sklearn.model_selection import GridSearchCV

# Définir le modèle de Random Forest Regression
rf = RandomForestRegressor()

# Définir la grille de paramètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['mse', 'mae']
}

# Effectuer la recherche d'hyperparamètres par validation croisée et recherche en grille
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés et leur performance
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleure performance : ", np.sqrt(-grid_search.best_score_))

# Utiliser les meilleurs paramètres pour ajuster le modèle final
rf_final = grid_search.best_estimator_
#rf_final = RandomForestRegressor(**grid_search.best_params_)
#rf_final.fit(X_train, y_train)

# Prédiction des valeurs de la variable cible pour les données de test
y_pred = rf_final.predict(X_test)

mse, rmse, r2 = performance(y_test, y_pred)

Meilleurs paramètres :  {'criterion': 'mse', 'max_depth': 20, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Meilleure performance :  0.4181109482885112
MSE : 0.22
RMSE: 0.47
R2 : 0.68


### 3.3.4. Ensembliste parallèle: Bagging Regression avec Arbre de décision

In [71]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# Division des données en ensembles d'apprentissage et de test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Construction du modèle de bagging avec 10 arbres de décision
bagging = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                            n_estimators=10, random_state=42)

# Apprentissage du modèle sur les données d'apprentissage
bagging.fit(X_train, y_train)

# Prédiction des valeurs de la variable cible pour les données de test
y_pred = bagging.predict(X_test)

# Evaluation de la précision des prévisions
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 0.24
RMSE: 0.49
R2 : 0.65


In [ ]:
Hyper paramètres pour Bagging

In [72]:
# Définir le modèle de base  (running time = 10s)
base_model = DecisionTreeRegressor()

# Définir le modèle Bagging avec GridSearchCV
bagging_model = GridSearchCV(BaggingRegressor(base_estimator=base_model),
                             param_grid={'n_estimators': [10, 100, 500]},
                             cv=5)

# Adapter le modèle sur les données d'entraînement
bagging_model.fit(X_train, y_train)

# Obtenir les meilleurs hyper-paramètres et le score R2 correspondant
print("Meilleurs hyper-paramètres : ", bagging_model.best_params_)
print("Score R2 : ", bagging_model.best_score_)

# Utiliser les meilleurs paramètres pour ajuster le modèle final
bagging_final = bagging_model.best_estimator_
#bagging_final = RandomForestRegressor(**bagging_model.best_params_)
#bagging_final.fit(X_train, y_train)

# Prédiction des valeurs de la variable cible pour les données de test
y_pred = bagging_final.predict(X_test)

mse, rmse, r2 = performance(y_test, y_pred)

Meilleurs hyper-paramètres :  {'n_estimators': 500}
Score R2 :  0.7456450837318108
MSE : 0.22
RMSE: 0.47
R2 : 0.67


### 3.3.5 Ensembliste séquentielle : GBoostRegression

GBoost Model 

In [73]:
from sklearn.ensemble import GradientBoostingRegressor

# Division des données en ensembles d'apprentissage et de test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialiser le modèle Gradient Boosting Regressor
gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Entraîner le modèle sur l'ensemble de données d'entraînement
gb_reg.fit(X_train, y_train)

# Prédire les valeurs de sortie sur l'ensemble de données de test
y_pred = gb_reg.predict(X_test)

# Calculer l'erreur quadratique moyenne (MSE) du modèle
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 0.19
RMSE: 0.43
R2 : 0.72


Super parametres pour GBoost

In [75]:
# Définir le modèle GBoostRegression avec GridSearchCV   (running time = 10 min , CPU at 30%)
gboost_model = GridSearchCV(GradientBoostingRegressor(),
                            param_grid={'n_estimators': [50, 100, 200],
                                        'max_depth': [2, 4, 6],
                                        'learning_rate': [0.01, 0.1, 1],
                                        'subsample': [0.5, 0.8, 1.0],
                                        'loss': ['ls', 'lad', 'huber']},
                            cv=5)

# Adapter le modèle sur les données d'entraînement
gboost_model.fit(X_train, y_train)

# Obtenir les meilleurs hyper-paramètres et le score R2 correspondant
print("Meilleurs hyper-paramètres : ", gboost_model.best_params_)
print("Score R2 : ", gboost_model.best_score_)

# Utiliser les meilleurs paramètres pour ajuster le modèle final
gboost_final = gboost_model.best_estimator_
#gboost_final = GradientBoostingRegressor(**gboost_model.best_params_)
#gboost_final.fit(X_train, y_train)

# Prédiction des valeurs de la variable cible pour les données de test
y_pred = gboost_final.predict(X_test)

mse, rmse, r2 = performance(y_test, y_pred)

Meilleurs hyper-paramètres :  {'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.5}
Score R2 :  0.7869810962914132
MSE : 0.18
RMSE: 0.42
R2 : 0.73


### 3.3.6 Arbre de décision 

In [74]:
from sklearn.tree import DecisionTreeRegressor

# création d'un objet d'arbre de décision de régression
regressor = DecisionTreeRegressor(random_state=42)

# entraînement de l'arbre de décision sur les données d'entraînement
regressor.fit(X_train, y_train)

# prédiction des résultats de la variable dépendante sur les données de test
y_pred = regressor.predict(X_test)

# Calculer l'erreur quadratique moyenne (MSE) du modèle
mse, rmse, r2 = performance(y_test, y_pred)

MSE : 0.41
RMSE: 0.64
R2 : 0.39


In [ ]:
# création d'un objet d'arbre de décision de régression  ()
regressor = DecisionTreeRegressor(random_state=0)

# définition de la grille de recherche
param_grid = {'max_depth': [2, 4, 6, 8],
              'min_samples_split': [2, 4, 6, 8],
              'min_samples_leaf': [1, 2, 4, 6]}

# création d'un objet GridSearchCV pour la recherche sur grille
grid_search = GridSearchCV(regressor, param_grid=param_grid, cv=5)

# entraînement du modèle avec la recherche sur grille
grid_search.fit(X_train, y_train)

# affichage des meilleurs hyperparamètres
print(grid_search.best_params_)

# Utiliser les meilleurs paramètres pour ajuster le modèle final
grid_final = grid_search.best_estimator_

# Prédiction des valeurs de la variable cible pour les données de test
y_pred = grid_final.predict(X_test)

mse, rmse, r2 = performance(y_test, y_pred)

### 3.3.6  Bagging avec plusieurs modèles 

In [26]:
# Create base models

model2 = LinearRegression()
model1 = KernelRidge(alpha=0.1, kernel='rbf', gamma=0.1)

# Create an ensemble of models using bagging
bagging = BaggingRegressor(base_estimator=[model1, model2], n_estimators=10, random_state=42)

# Fit the bagging ensemble to the training data
bagging.fit(X_train_std, y_train)

# Evaluate the performance of the bagging ensemble on the test data
y_pred = bagging.predict(X_test_std)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mse, rmse, r2 = performance(y_test, y_pred)

AttributeError: 'list' object has no attribute 'fit'